<a href="https://colab.research.google.com/github/AdamPeetz/PlaneGAN/blob/main/MSDS696_DCGAN_V6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# example of a dcgan on cifar10
from numpy import zeros
from numpy import ones
from numpy.random import randn, random
from numpy.random import randint
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose, Conv3DTranspose
from keras.layers import LeakyReLU, ReLU
from keras.layers import Dropout
from keras.layers import BatchNormalization
from matplotlib import pyplot
import tensorflow as tf
import cv2
import numpy as np
import pathlib

import os, shutil 
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# define the standalone discriminator model
def define_discriminator(in_shape=(128,128,3)):
	init = RandomNormal(mean=0.0, stddev=0.02)
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 64x64
	model.add(Conv2D(98, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
	model.add(Dropout(0.4))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 32x32
	model.add(Conv2D(128, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
	model.add(Dropout(0.4))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 16x16
	model.add(Conv2D(192, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
	model.add(Dropout(0.4))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 8x8
	model.add(Conv2D(256, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
	model.add(Dropout(0.4))
	model.add(LeakyReLU(alpha=0.2))
 	# downsample to 4x4
	model.add(Conv2D(256, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
	model.add(Dropout(0.4))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model


In [11]:
# define the standalone generator model
def define_generator(latent_dim):
	init = RandomNormal(mean=0.0, stddev=0.02)
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
#	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(1024, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
#	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(512, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
#	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(264, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
#	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
 	# upsample to 64x64
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
#	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
 	# upsample to 128x128
	model.add(Conv2DTranspose(64, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# Param Expand
#	model.add(Conv2D(256, (1,1), kernel_initializer=init, padding='same'))
#	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [5]:
#load a custom image dataset 
# tensorflow documentation
def load_custom_image_set(dataroot,resizedim,directorylevels):
    #image parse function
    def parse_image(filename):
        parts = tf.strings.split(filename, os.sep)
        label = parts[-2]
        image = tf.io.read_file(filename)
        image = tf.io.decode_jpeg(image)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, [resizedim,resizedim])
        image = (image * 2) -1
        return image, label
    
    data_root = pathlib.Path(dataroot)
    if directorylevels == 1:
      list_ds = tf.data.Dataset.list_files(str(data_root/'*/*'))
    if directorylevels == 2:
      list_ds = tf.data.Dataset.list_files(str(data_root/'*/*/*'))
    images_ds = list_ds.map(parse_image)
    #https://stackoverflow.com/questions/70535683/extract-data-from-tensorflow-dataset-e-g-to-numpy
    images = np.asarray(list(images_ds.map(lambda x, y: x)))
    return images

# label smoothing function
def smooth_positive_labels(y):
	return y - 0.3 + (random(y.shape) * 0.5)
                  
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	# apply label smoothing
	y = smooth_positive_labels(y)
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# modified with suggestion from Das Shuvo (2020)
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	input_tensor = tf.convert_to_tensor(x_input)
	X = g_model(input_tensor)
	X = X.numpy()
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [6]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
#	example = (examples + 1) * 255
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = '/content/gdrive/My Drive/planegan/sample_output/generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()
 
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d1_hist, label='d-real')
	pyplot.plot(d2_hist, label='d-fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# plot discriminator accuracy
	pyplot.subplot(2, 1, 2)
	pyplot.plot(a1_hist, label='acc-real')
	pyplot.plot(a2_hist, label='acc-fake')
	pyplot.legend()
	# save plot to file
	pyplot.savefig('/content/gdrive/My Drive/planegan/saved_models/plot_line_plot_loss.png')
	pyplot.close()


In [7]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = '/content/gdrive/My Drive/planegan/saved_models/generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=300, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# define half batches for training
	half_batch = int(n_batch / 2)
  # prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss
			if (j+1) % 5 == 0:
				print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)
	 		# save plot to directory
			plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)
		# record training metrics
		if (i+1) % 2 == 0:
			d1_hist.append(d_loss1)
			d2_hist.append(d_loss2)
			g_hist.append(g_loss)
			a1_hist.append(d_acc1)
			a2_hist.append(d_acc2)

In [9]:
# load image data
image_path = "/content/gdrive/My Drive/planegan/airplanes_resized/"
resizedim = 128
directorylevels = 1
dataset = load_custom_image_set(image_path,resizedim,directorylevels)

In [12]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=600, n_batch=128)

>1, 5/60, d1=0.523, d2=0.702 g=0.694
>1, 10/60, d1=0.191, d2=0.638 g=0.798
>1, 15/60, d1=0.264, d2=0.219 g=2.147
>1, 20/60, d1=0.391, d2=0.050 g=6.890
>1, 25/60, d1=0.544, d2=0.023 g=4.763
>1, 30/60, d1=0.418, d2=0.055 g=4.427
>1, 35/60, d1=0.528, d2=0.031 g=3.714
>1, 40/60, d1=0.332, d2=0.067 g=3.563
>1, 45/60, d1=0.457, d2=0.080 g=3.270
>1, 50/60, d1=0.209, d2=0.053 g=3.560
>1, 55/60, d1=0.391, d2=0.084 g=2.773
>1, 60/60, d1=0.325, d2=0.064 g=3.466
>2, 5/60, d1=0.455, d2=0.106 g=9.192
>2, 10/60, d1=0.447, d2=0.086 g=2.831
>2, 15/60, d1=0.597, d2=0.713 g=0.774
>2, 20/60, d1=0.528, d2=0.730 g=0.693
>2, 25/60, d1=0.834, d2=0.546 g=0.905
>2, 30/60, d1=0.706, d2=0.365 g=3.481
>2, 35/60, d1=0.714, d2=0.241 g=2.517
>2, 40/60, d1=0.361, d2=0.072 g=3.366
>2, 45/60, d1=0.480, d2=0.048 g=4.144
>2, 50/60, d1=0.186, d2=0.023 g=4.843
>2, 55/60, d1=0.631, d2=0.555 g=1.327
>2, 60/60, d1=0.589, d2=0.646 g=1.915
>3, 5/60, d1=0.385, d2=1.014 g=1.350
>3, 10/60, d1=0.787, d2=0.351 g=2.128
>3, 15/60, d1=0

>11, 5/60, d1=0.720, d2=0.763 g=0.977
>11, 10/60, d1=0.624, d2=0.585 g=1.068
>11, 15/60, d1=0.681, d2=0.506 g=1.189
>11, 20/60, d1=0.663, d2=0.482 g=1.233
>11, 25/60, d1=0.506, d2=0.676 g=1.206
>11, 30/60, d1=0.675, d2=0.523 g=1.258
>11, 35/60, d1=0.570, d2=0.590 g=1.122
>11, 40/60, d1=0.716, d2=0.591 g=1.042
>11, 45/60, d1=0.689, d2=0.583 g=1.055
>11, 50/60, d1=0.799, d2=0.500 g=1.150
>11, 55/60, d1=0.805, d2=0.662 g=1.102
>11, 60/60, d1=0.798, d2=0.469 g=1.178
>12, 5/60, d1=0.664, d2=0.720 g=0.821
>12, 10/60, d1=0.652, d2=0.654 g=0.847
>12, 15/60, d1=0.652, d2=0.666 g=0.881
>12, 20/60, d1=0.703, d2=0.563 g=0.865
>12, 25/60, d1=0.728, d2=0.519 g=1.093
>12, 30/60, d1=0.818, d2=0.549 g=1.068
>12, 35/60, d1=0.658, d2=0.771 g=0.810
>12, 40/60, d1=0.614, d2=0.663 g=0.800
>12, 45/60, d1=0.535, d2=0.587 g=1.004
>12, 50/60, d1=0.447, d2=0.733 g=0.935
>12, 55/60, d1=0.768, d2=0.563 g=1.025
>12, 60/60, d1=0.767, d2=0.576 g=1.108
>13, 5/60, d1=0.765, d2=0.486 g=1.142
>13, 10/60, d1=0.815, d2=0.6

>21, 5/60, d1=0.693, d2=0.609 g=0.949
>21, 10/60, d1=0.757, d2=0.459 g=1.192
>21, 15/60, d1=0.656, d2=0.640 g=0.867
>21, 20/60, d1=0.640, d2=0.648 g=0.895
>21, 25/60, d1=0.682, d2=0.543 g=1.076
>21, 30/60, d1=0.676, d2=0.560 g=1.077
>21, 35/60, d1=0.761, d2=0.628 g=1.048
>21, 40/60, d1=0.585, d2=0.551 g=1.072
>21, 45/60, d1=0.756, d2=0.604 g=0.970
>21, 50/60, d1=0.567, d2=0.734 g=0.914
>21, 55/60, d1=0.798, d2=0.608 g=0.961
>21, 60/60, d1=0.648, d2=0.537 g=1.195
>22, 5/60, d1=0.791, d2=0.569 g=1.263
>22, 10/60, d1=0.679, d2=0.656 g=0.937
>22, 15/60, d1=0.860, d2=0.572 g=1.146
>22, 20/60, d1=0.750, d2=0.637 g=1.038
>22, 25/60, d1=0.815, d2=0.531 g=0.985
>22, 30/60, d1=0.658, d2=0.735 g=0.978
>22, 35/60, d1=0.765, d2=0.525 g=1.109
>22, 40/60, d1=0.615, d2=0.506 g=0.966
>22, 45/60, d1=0.690, d2=0.620 g=1.110
>22, 50/60, d1=0.752, d2=0.693 g=1.025
>22, 55/60, d1=0.815, d2=0.564 g=0.932
>22, 60/60, d1=0.790, d2=0.563 g=0.943
>23, 5/60, d1=0.681, d2=0.655 g=0.853
>23, 10/60, d1=0.663, d2=0.6

>31, 5/60, d1=0.708, d2=0.512 g=1.254
>31, 10/60, d1=0.613, d2=0.519 g=1.329
>31, 15/60, d1=0.600, d2=0.575 g=1.397
>31, 20/60, d1=0.563, d2=0.482 g=1.368
>31, 25/60, d1=0.719, d2=0.493 g=1.442
>31, 30/60, d1=0.871, d2=0.616 g=1.601
>31, 35/60, d1=1.011, d2=0.593 g=1.211
>31, 40/60, d1=0.645, d2=0.517 g=1.271
>31, 45/60, d1=0.682, d2=0.559 g=1.270
>31, 50/60, d1=0.636, d2=0.858 g=1.371
>31, 55/60, d1=0.710, d2=0.476 g=1.244
>31, 60/60, d1=0.562, d2=1.120 g=1.510
>32, 5/60, d1=0.618, d2=0.660 g=1.305
>32, 10/60, d1=0.711, d2=0.533 g=0.985
>32, 15/60, d1=0.600, d2=0.572 g=1.311
>32, 20/60, d1=0.757, d2=0.579 g=1.322
>32, 25/60, d1=0.873, d2=0.397 g=1.209
>32, 30/60, d1=0.929, d2=0.400 g=1.412
>32, 35/60, d1=0.623, d2=0.585 g=1.178
>32, 40/60, d1=0.642, d2=0.492 g=1.162
>32, 45/60, d1=0.847, d2=0.538 g=1.170
>32, 50/60, d1=0.684, d2=0.560 g=1.123
>32, 55/60, d1=0.635, d2=0.501 g=1.439
>32, 60/60, d1=0.588, d2=0.795 g=1.272
>33, 5/60, d1=0.511, d2=0.546 g=1.304
>33, 10/60, d1=0.592, d2=0.5

>41, 5/60, d1=0.593, d2=0.628 g=1.541
>41, 10/60, d1=0.672, d2=0.560 g=1.256
>41, 15/60, d1=0.628, d2=0.561 g=1.341
>41, 20/60, d1=0.637, d2=0.531 g=1.394
>41, 25/60, d1=0.702, d2=0.405 g=1.313
>41, 30/60, d1=0.528, d2=0.470 g=1.479
>41, 35/60, d1=0.816, d2=0.497 g=1.475
>41, 40/60, d1=0.632, d2=0.585 g=1.437
>41, 45/60, d1=0.533, d2=0.700 g=1.484
>41, 50/60, d1=0.650, d2=0.865 g=1.706
>41, 55/60, d1=0.911, d2=0.345 g=1.354
>41, 60/60, d1=0.726, d2=0.420 g=1.256
>42, 5/60, d1=0.617, d2=0.463 g=1.173
>42, 10/60, d1=0.637, d2=0.493 g=1.243
>42, 15/60, d1=0.837, d2=0.344 g=1.375
>42, 20/60, d1=0.670, d2=0.501 g=1.427
>42, 25/60, d1=0.577, d2=0.422 g=1.682
>42, 30/60, d1=0.557, d2=0.445 g=1.543
>42, 35/60, d1=0.513, d2=0.663 g=1.599
>42, 40/60, d1=0.567, d2=0.728 g=1.590
>42, 45/60, d1=0.656, d2=0.491 g=1.422
>42, 50/60, d1=0.801, d2=0.743 g=1.101
>42, 55/60, d1=0.878, d2=0.673 g=1.097
>42, 60/60, d1=0.809, d2=0.650 g=0.863
>43, 5/60, d1=0.757, d2=0.558 g=0.996
>43, 10/60, d1=0.695, d2=0.6

...

>311, 5/60, d1=0.632, d2=0.438 g=1.708
>311, 10/60, d1=0.706, d2=0.467 g=1.499
>311, 15/60, d1=0.593, d2=0.382 g=1.744
>311, 20/60, d1=0.623, d2=0.408 g=1.516
>311, 25/60, d1=0.498, d2=0.336 g=1.635
>311, 30/60, d1=0.681, d2=0.406 g=1.703
>311, 35/60, d1=0.647, d2=0.346 g=1.701
>311, 40/60, d1=0.714, d2=0.338 g=1.536
>311, 45/60, d1=0.609, d2=0.507 g=1.597
>311, 50/60, d1=0.652, d2=0.434 g=1.586
>311, 55/60, d1=0.667, d2=0.377 g=1.663
>311, 60/60, d1=0.674, d2=0.327 g=1.558
>312, 5/60, d1=0.536, d2=0.387 g=1.519
>312, 10/60, d1=0.681, d2=0.443 g=1.582
>312, 15/60, d1=0.675, d2=0.392 g=1.581
>312, 20/60, d1=0.520, d2=0.422 g=1.930
>312, 25/60, d1=0.547, d2=0.486 g=1.605
>312, 30/60, d1=0.543, d2=0.625 g=1.673
>312, 35/60, d1=0.575, d2=0.531 g=1.481
>312, 40/60, d1=0.627, d2=0.380 g=1.793
>312, 45/60, d1=0.609, d2=0.392 g=1.522
>312, 50/60, d1=0.593, d2=0.371 g=1.672
>312, 55/60, d1=0.576, d2=0.365 g=1.562
>312, 60/60, d1=0.702, d2=0.389 g=1.619
>313, 5/60, d1=0.648, d2=0.480 g=1.689
>31

>321, 5/60, d1=0.582, d2=0.476 g=1.651
>321, 10/60, d1=0.620, d2=0.363 g=1.743
>321, 15/60, d1=0.635, d2=0.544 g=1.726
>321, 20/60, d1=0.622, d2=0.275 g=1.787
>321, 25/60, d1=0.516, d2=0.510 g=1.714
>321, 30/60, d1=0.516, d2=0.477 g=1.531
>321, 35/60, d1=0.769, d2=0.437 g=1.803
>321, 40/60, d1=0.565, d2=0.439 g=1.542
>321, 45/60, d1=0.534, d2=0.461 g=1.590
>321, 50/60, d1=0.305, d2=0.467 g=1.567
>321, 55/60, d1=0.517, d2=0.390 g=1.665
>321, 60/60, d1=0.557, d2=0.256 g=1.910
>322, 5/60, d1=0.523, d2=0.382 g=1.670
>322, 10/60, d1=0.547, d2=0.383 g=2.129
>322, 15/60, d1=0.563, d2=0.401 g=1.871
>322, 20/60, d1=0.630, d2=0.419 g=1.811
>322, 25/60, d1=0.693, d2=0.510 g=1.601
>322, 30/60, d1=0.462, d2=0.481 g=2.044
>322, 35/60, d1=0.603, d2=0.355 g=1.648
>322, 40/60, d1=0.739, d2=0.355 g=1.745
>322, 45/60, d1=0.599, d2=0.397 g=1.784
>322, 50/60, d1=0.586, d2=0.466 g=1.736
>322, 55/60, d1=0.757, d2=0.305 g=1.656
>322, 60/60, d1=0.529, d2=0.359 g=1.608
>323, 5/60, d1=0.652, d2=0.557 g=1.711
>32

>331, 5/60, d1=0.561, d2=0.383 g=1.871
>331, 10/60, d1=0.439, d2=0.568 g=1.729
>331, 15/60, d1=0.558, d2=0.399 g=1.733
>331, 20/60, d1=0.677, d2=0.442 g=1.477
>331, 25/60, d1=0.732, d2=0.392 g=1.744
>331, 30/60, d1=0.400, d2=0.345 g=1.733
>331, 35/60, d1=0.642, d2=0.420 g=1.620
>331, 40/60, d1=0.602, d2=0.377 g=1.724
>331, 45/60, d1=0.490, d2=0.443 g=1.832
>331, 50/60, d1=0.666, d2=0.355 g=1.990
>331, 55/60, d1=0.728, d2=0.319 g=1.869
>331, 60/60, d1=0.725, d2=0.344 g=1.686
>332, 5/60, d1=0.545, d2=0.335 g=2.025
>332, 10/60, d1=0.625, d2=0.557 g=2.165
>332, 15/60, d1=0.593, d2=0.305 g=2.033
>332, 20/60, d1=0.757, d2=0.464 g=1.836
>332, 25/60, d1=0.569, d2=0.393 g=1.746
>332, 30/60, d1=0.724, d2=0.301 g=1.648
>332, 35/60, d1=0.576, d2=0.455 g=1.522
>332, 40/60, d1=0.565, d2=0.431 g=1.721
>332, 45/60, d1=0.631, d2=0.509 g=1.878
>332, 50/60, d1=0.495, d2=0.407 g=1.729
>332, 55/60, d1=0.633, d2=0.277 g=1.859
>332, 60/60, d1=0.732, d2=0.265 g=1.429
>333, 5/60, d1=0.675, d2=0.437 g=1.817
>33

>341, 5/60, d1=0.605, d2=0.492 g=1.819
>341, 10/60, d1=0.745, d2=0.564 g=1.761
>341, 15/60, d1=0.645, d2=0.395 g=1.540
>341, 20/60, d1=0.577, d2=0.371 g=1.627
>341, 25/60, d1=0.626, d2=0.468 g=1.571
>341, 30/60, d1=0.652, d2=0.396 g=1.662
>341, 35/60, d1=0.621, d2=0.423 g=1.666
>341, 40/60, d1=0.667, d2=0.362 g=1.807
>341, 45/60, d1=0.580, d2=0.496 g=1.652
>341, 50/60, d1=0.615, d2=0.447 g=1.687
>341, 55/60, d1=0.714, d2=0.424 g=1.757
>341, 60/60, d1=0.599, d2=0.414 g=1.788
>342, 5/60, d1=0.619, d2=0.506 g=1.604
>342, 10/60, d1=0.727, d2=0.433 g=1.745
>342, 15/60, d1=0.609, d2=0.333 g=1.767
>342, 20/60, d1=0.489, d2=0.387 g=1.924
>342, 25/60, d1=0.548, d2=0.490 g=1.734
>342, 30/60, d1=0.574, d2=0.357 g=1.445
>342, 35/60, d1=0.436, d2=0.510 g=1.832
>342, 40/60, d1=0.639, d2=0.422 g=1.817
>342, 45/60, d1=0.574, d2=0.452 g=1.664
>342, 50/60, d1=0.663, d2=0.406 g=1.605
>342, 55/60, d1=0.511, d2=0.548 g=1.539
>342, 60/60, d1=0.471, d2=0.528 g=1.720
>343, 5/60, d1=0.612, d2=0.361 g=1.980
>34

KeyboardInterrupt: ignored

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# create and save a plot of generated images
def save_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :])
	pyplot.show()

# load model
model = gan_model
# generate images
latent_points = generate_latent_points(100, 100)
# generate images
X = model.predict(latent_points)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, 10)

# References

Brownlee, Jason. (2019). Generative Adversarial Networks with Python. Ebook. Machine Learning Mastery. 

Das Shuvo, Falguni. (2020). Repeatedly calling model.predict(...) results in memory leak. GitHub. Retrieved 3/15/2023 from https://github.com/keras-team/keras/issues/13118